## Web Application with Django 3
### Building a Bookstore Application -- version 2

In today's class, we will continue building our online Bookstore application from where we stopped in week 5.

We wiil be adding the following features to the app and finally deploying it on `render`.

- Search 
- Pagination 
- Online Payment Integration
- User Permissions & Authorisation
- Email SMTP integration (Google Mail)
- Whitenoise Static files and Security Setup
- Github and Webapplication Deployment with CI/CD

#### Django Search
**Search** is an essential aspect of the majority of websites, especially those in e-commerce, such as our Bookstore Application. In this section, we will discover how to integrate primary search functionalities using ***forms and filters***.

Subsequently, we will enhance it with advanced logic and briefly discuss methods to delve further into search capabilities within Django.

Let's begin by creating the search results page. This involves setting up a specific web address (URL), a function to handle the page's logic (view), and a design template. 

The order in which we do this doesn't really make a big difference, but we'll follow this sequence. 
In the file named` book/urls.py`, include a path for `search/` with a URL name `search_results`, which will be connected to a view named `SearchResultsListView.`

In [ ]:
# books/urls.py

from django.urls import path
from .views import BookListView, BookDetailView, SearchResultsListView # new


urlpatterns = [ 
...
path("search/", SearchResultsListView.as_view(), name="search_results"), # new

]

Next, we'll work on the view called `SearchResultsListView`. Initially, it will display a list of all the books available. This is a perfect situation to use `ListView`, a built-in feature. 

We'll create a template called` search_results.html`, which will be placed in the` templates/book/` directory. 

The only new code we need to add is for `SearchResultsListView`, assuming that we've already imported ListView and the Book model at the beginning of the file.

In [ ]:
# books/views.py

...
class SearchResultsListView(ListView): # new
    model = Book
    context_object_name = "book_list"
    template_name = "book/search_results.html"


In [ ]:
{% extends "base.html" %}
{% load static %}

{% block content %}

		<!--************************************
				Main Start
		*************************************-->
		<main id="tg-main" class="tg-main tg-haslayout">
			<!--************************************
					Best Selling Start
			*************************************-->
			<section class="tg-sectionspace tg-haslayout">
				<div class="container">
					<div class="row">
						<div class="col-xs-12 col-sm-12 col-md-12 col-lg-12">
							<div class="tg-sectionhead">
								<h2><span>Search</span>Results</h2>
								<a class="tg-btn" href="javascript:void(0);">View All</a>
							</div>
						</div>
						<div class="col-xs-12 col-sm-12 col-md-12 col-lg-12">
							<div id="tg-bestsellingbooksslider" class="tg-bestsellingbooksslider tg-bestsellingbooks owl-carousel">
                            {% for book in book_list %}
								<div class="item">
									<div class="tg-postbook">
										<figure class="tg-featureimg">
											<div class="tg-bookimg">
												<div class="tg-frontcover"><img src="{{ book.cover.url }}" alt="image description"></div>
												<div class="tg-backcover"><img src="{{book.cover.url}}" alt="image description"></div>
											</div>
											
										</figure>
										<div class="tg-postbookcontent">
											<ul class="tg-bookscategories">
                                             {% for genre in book.genre.all%}
												<li><a href="javascript:void(0);">{{genre}}</a></li>
                                                {%endfor %}
											</ul>
											<div class="tg-themetagbox"><span class="tg-themetag">sale</span></div>
											<div class="tg-booktitle">
												<h3><a href="javascript:void(0);">{{book.title}}</a></h3>
											</div>
											<span class="tg-bookwriter">By: <a href="javascript:void(0);">{{book.author}}</a></span>
											
											<span class="tg-bookprice">
												<ins>{{book.price}}</ins>
												<del>$27.20</del>
											</span>
											<a class="tg-btn tg-btnstyletwo" href="{% url 'book_detail' book.pk %}">
												<i class="bi bi-eye"></i>
												<em>View Details</em>
											</a>
										</div>
									</div>
								</div>
								{% endfor %}
								
							</div>
						</div>
					</div>
				</div>
			</section>
			<!--************************************
					Best Selling End
			*************************************-->
		
		
		
		
			
			
		</main>
		<!--************************************
				Main End
		*************************************-->
{% endblock content %}

##### Search Form
Let’s add a basic search form to the base template so it is available on every page. That means
updating the base.html template.

In [ ]:
<!-- templates/base.html -->

<form class="tg-formtheme tg-formsearch" action="{% url 'search_results' %}" method="get"> 
    <fieldset>

        <input type="search" name="q" class="typeahead form-control" placeholder="Search by title or author's name"> 

        <button type="submit" class="tg-btn">Search</button>

        
    </fieldset>
    <a href="javascript:void(0);">+  Advanced Search</a>
</form>

In [ ]:
# books/views.py
from django.db.models import Q


class SearchResultsListView(ListView):

    ...
    def get_queryset(self): # new
        query = self.request.GET.get("q")
        
        return Book.objects.filter(Q(title__icontains=query))


#### Django Pagination
Django provides high-level and low-level ways to help you manage paginated data – that is, data that’s split across several pages, with “Previous/Next” links.

##### The Paginator class
Under the hood, all methods of pagination use the` Paginator` class. It does all the heavy lifting of actually splitting a `QuerySet` into `Page objects`.

##### Paginating a ListView
`django.views.generic.list.ListView` provides a builtin way to paginate the displayed list. You can do this by adding a `paginate_by` attribute to your view class.

for example:



In [ ]:
#This is just an example(Dont add to your code)

from django.views.generic import ListView
from myapp.models import Contact


class ContactListView(ListView):
    paginate_by = 2
    model = Contact

This limits the number of objects per page and adds a `paginator` and `page_obj` to the `context`. To allow your users to navigate between pages, add links to the` next` and` previous` page, in your template like this:

In [ ]:
<!-- templates/home.html -->

<div class="container-fluid">
    <diV class="row">
        <div class="col-12 ">
            {% if is_paginated %}
          <div class="card">
              <div class="card-body mx-auto">
              <nav aria-label="Page navigation example">
                  <ul class="pagination">
                      {% if page_obj.has_previous %}
                      <li class="page-item"><a class="page-link" href="?page={{page_obj.prebious_page_number}}">Previous</a></li>
                      {% endif %}
                      {% for number in page_obj.paginator.page_range %}
                      <li class="page-item"><a class="page-link" href="?page={{number}}">{{number}}</a></li>
                      {% endfor %}
                      {% if page_obj.has_next %}
                      <li class="page-item"><a class="page-link" href="?page={{page_obj.prebious_page_number}}">Next</a></li>
                      {% endif %}
                  </ul>
              </nav>
          </div>
          
          {% endif %}
        </div>
    </div>
</diV>

#### Payment Integration (Monnify)

In this section, we will be integrating payments into our bookstore application. This allows users to make payment directly from our website using their card, USSD, tranfers in collaborating with payment infrastructure companies etc

There are many payment provider we can use to initiate online payments. Some of the providers are listed as follows;

- Stripe
- Paypal
- Paystack
- Flutterwave
- Monnify
- braintree etc

For this bookstore app will be integrating with `monnify`. They are work in similar ways so it doesn't matter which provider we chose to use

##### Bellow are the steps to integrating monnify payment into our App

- Visit the monnify website and create an account.
- In your dashboard, Under the developer tab, you wiil see your Test API and SECRET KEY
- Copy them in your Django project` settings.py` file as Settings variable
- Next, Install the Monnify Checkout Page to our website 

Lets add the following code into our project step by step

In [ ]:
# bookstore_settings.py

MONNIFY_SECRET_KEY= env("MONNIFY_SECRET_KEY")
MONNIFY_API_KEY = env("MK_TEST_FA25GM45T3")

In [ ]:
# book/views.py
from bookstore_project.settings import MONNIFY_SECRET_KEY, MONNIFY_API_KEY 



class HomeDetailView(DetailView):
    
    ...
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        queryset = Book.objects.all()
           
        context.update({
                "book_list": queryset,
                "MONNIFY_SECRET_KEY": MONNIFY_SECRET_KEY,
                "MONNIFY_API_KEY": MONNIFY_API_KEY
                
            })
        return context

In [ ]:
<!-- templates/base.html -->

<script type="text/javascript" src="https://sdk.monnify.com/plugin/monnify.js"></script>
<script src="https://code.jquery.com/jquery-3.1.1.min.js"> 

In [ ]:
<!-- templates/book/book_detail.html -->


<script>

    // Generate A CSRF Cookie
    function getCookie(name) {
        let cookieValue = null;
        if (document.cookie && document.cookie !== '') {
            const cookies = document.cookie.split(';');
            for (let i = 0; i < cookies.length; i++) {
                const cookie = cookies[i].trim();
                // Does this cookie string begin with the name we want?
                if (cookie.substring(0, name.length + 1) === (name + '=')) {
                    cookieValue = decodeURIComponent(cookie.substring(name.length + 1));
                    break;
                    }
                }
            }
            return cookieValue;
        }       
    const csrftoken = getCookie('csrftoken');
    console.log('csrftoken===>', csrftoken)


    // Get Data from Django detail page
    var grand_total = "{{ book.price }}"
   
   
      
    // MONNIFY FUNCTION : Handles the payment process

    var paymentDiv = document.getElementById('paymentDiv');  // Get the clicked element
    paymentDiv.addEventListener('submit', payWithMonnify, false); // Append an event listener
        function payWithMonnify() {                               // Call Monnify Api
            MonnifySDK.initialize({
                amount: grand_total,
                currency: "NGN",
                reference: new String((new Date()).getTime()),
                customerFullName: "{{user.username}}",
                customerEmail: "{{user.email}}",
                apiKey: "{{MONNIFY_API_KEY}}",
                contractCode: "0680159477",
                paymentDescription: "payment for {{book.title}} book",
                metadata: {
                    "name": "{{book.title}}",
                },
                onLoadStart: () => {
                    console.log("loading has started");
                },
                onLoadComplete: () => {
                    console.log("SDK is UP");
                },

                onComplete: (response) => {

                    //Implement what happens when the transaction is completed.
                    let transaction_id = response.paymentReference;
                    let status = response.status
                    console.log(response); // log response to console
                    
                    const element = document.getElementById('monnify-button');
                    element.innerHTML = '';
                    element.innerHTML = '<h6 class="text-center text-white" ><i class="fa fa-spinner fa-spin"></i> Please wait...</h6>';
                    
                    // Invoke the Redirect Function
                    redirect(response.status)
                    
                },
                onClose: function(data) {

                    //Implement what should happen when the modal is closed here
                    console.log(data);
                    alert("Payment Cancelled")
                }
            });
        }


    // Function to Handle Alert and Page Redirect after payment

    function redirect(response_status = "SUCCESS") {

        if (response_status === "SUCCESS") {

            alert(response_status);
            window.location.href = "https://drive.google.com/file/d/1-ehG-4yk6hfdtqmo2-0DydSJxClYtlJy/view?usp=sharing"; //Downloadpage Upon Successful

        } else {
            alert(response_status);
            window.location.href = "{% url 'book_detail' book.pk %}"; // Book Detail page when unsuccessful
        }
    }


</script>


#### Authorisation

Note that authorisation is different than authentication which is the process of registering
and logging-in users.

Authorization restricts access; authentication enables a user sign up and log in flow
Now we will like to restrict users of our bookstore app to access to the book detail page or screen. 

To do this we will make use of django `Mixins`; which is a special kind of multiple inheritance that Django uses to avoid duplicate code and still allow customization.

To restrict detail view access to only logged in users, Django has a `LoginRequiredmixin` that we can use. 

In the `book/views.py` file, import `LoginRequiredMixin` at the top and then it add to the
`BookDetailView`.

 Make sure that the mixin is to the left of `DetailView` so it will be read first.

In [ ]:
# book/views.py

from django.contrib.auth.mixins import LoginRequiredMixin

class HomeDetailView(LoginRequiredMixin, DetailView):
    model = Book
    template_name= "book/book_detail.html"
   
    
    
    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        queryset = Book.objects.all()
           
        context.update({
                "book_list": queryset,
                "MONNIFY_SECRET_KEY": MONNIFY_SECRET_KEY,
                "MONNIFY_API_KEY": MONNIFY_API_KEY,
                # "FLWPUBK_TEST": FLWPUBK_TEST
                
            })
        return context


#### GMAIL SMTP integration

SMTP stands for ***Simple Mail Transfer Protocol*** (SMTP)

SMTP is the email protocol that allows you to send messages from one email account to one or more email addresses. Without it, email communication would not exist. Why?

An email protocol is a combination of parameters and rules for the exchange of information between email accounts. SMTP is used to communicate between servers and establish which servers receive or route your messages. It’s the most commonly used protocol for outgoing emails.

To configure Gmail as a provide, We take the following step to generate an app password from google.

- Make sure you enable 2FA Account in your Google acoount
- Log into your Google Account
- Follow this link https://myaccount.google.com/apppasswords to create an App Passowrd
- Next past your email address and app password into the config settting below in your `settings.py` file

In [ ]:
# bookstore_project


# EMAIL_BACKEND = "django.core.mail.backends.console.EmailBackend"
EMAIL_BACKEND = 'django.core.mail.backends.smtp.EmailBackend'

EMAIL_HOST = "smtp.gmail.com"
EMAIL_PORT = "587"
EMAIL_USE_TLS = True
EMAIL_HOST_USER ="jamezslim90@gmail.com"
EMAIL_HOST_PASSWORD="boiyhaxzidarokpp"
DEFAULT_FROM_EMAIL = 'BookStore Online  <jamezslim90@gmail.com>'
                       

And we're done. Our app will be now be able to send email notifications to users e.g during sign up and forget password flow.

#### Pushing to GitHub

Next is we push our code to GitHub using the following Commands and the desktop interface.

In [ ]:
git init
git status
git add .
git commit -m "Initial Commit"


#### Deploying Django on Render
To prepare our Django project for production on `Render,` we’ll make a couple adjustments to its configuration:

##### Setting up Static Files
- Add WhiteNoise as a dependency

In [ ]:
pip install whitenoise
pip freeze > requirements.txt

- Open `settings.py` in your project’s main directory 

Add the following to the `MIDDLEWARE` list, immediately after `SecurityMiddleware`

In [ ]:
MIDDLEWARE = [
    'django.middleware.security.SecurityMiddleware',
    'whitenoise.middleware.WhiteNoiseMiddleware', #new
    ...
]

Make the following modifications in `bookstore/settings.py`

In [ ]:

    
STATICFILES_DIRS = [BASE_DIR / "static"]

# http://whitenoise.evans.io/en/stable/django.html#add-compression-and-caching-support
# STATICFILES_STORAGE = "whitenoise.storage.CompressedManifestStaticFilesStorage"
STATICFILES_STORAGE = "whitenoise.storage.CompressedStaticFilesStorage"
STATIC_ROOT = BASE_DIR / "staticfiles"
WHITENOISE_MANIFEST_STRICT = False

##### Create a Build Script

- Create a new file called `build.sh` in your project’s root directory and paste in the following:

In [ ]:
#!/usr/bin/env bash
# Exit on error
set -o errexit

# Modify this line as needed for your package manager (pip, poetry, etc.)
pip install -r requirements.txt

# Convert static asset files
python manage.py collectstatic --no-input

# Apply any outstanding database migrations
python manage.py migrate


- We’ll run your project with `Uvicorn` and `Gunicorn`. Add these dependencies to your project:

In [ ]:
pip install gunicorn uvicorn
pip freeze > requirements.txt

- Try running your project locally!

In [ ]:
python -m gunicorn bookstore_project.asgi:application -k uvicorn.workers.UvicornWorker

- Commit all changes and push them to your repository. Your project is ready to deploy to Render!

##### Create A Manual Deploy on Render

- `Go to the render.com and create an account. Navaigate to dashboard`
- Create a new web service on Render, pointing it to your project’s GitHub/GitLab repository (give Render permission to access it if you haven’t already).
- Select Python for the runtime and set the following properties (replace mysite with your project’s name):


In [ ]:
Build Command :  sh build.sh
Start Command : gunicorn bookstore_project.wsgi